In [3]:
import numpy as np
import openmc
import pandas as pd

In [4]:
df = pd.read_csv('Tanks_Slurry_Inventory - all_tank_data.csv')

In [141]:
def total_molecular_mass_per_molecule_dict(data,tankID,WastePhase,compounds,molecules):
    c = 12.011
    h = 1.008
    o = 15.999
    p = 30.973761998
    n = 14.007
    cl = 35.45
    f = 18.998403162
    s = 32.06
    tl = 204.38
    nuclides = ['C','H','O','P','N','Cl','F','S','Tl']
    elem_mass = np.array([c,h,o,p,n,cl,f,s,tl])
    compounds = compounds
    compound_mol_dict = dict(zip(compounds,molecules))
    total_compound_molecules = np.zeros((9))
    analytes = data.loc[(data['WasteSiteId'] == tankID) & (data['WastePhase'] == WastePhase),['Analyte','WastePhase Volume (L)']].values[:,0]
    for compound in analytes:
        if compound in compounds:
           total_compound_molecules += compound_mol_dict[compound]
    total_molecule_mass = total_compound_molecules*elem_mass
    nuclide_per_compound = dict(zip(nuclides,total_molecule_mass))
    return nuclide_per_compound


'''
Takes in the total excel waste data, tank ID, desired waste phase, and the list of total individual elements surveyed and
outputs the mass of each element surveyed per tank per waste phase as a dictionary
'''
def element_masses_per_tank_per_waste_phase(data,tankID,WastePhase,elements):
    analytes = df.loc[(df['WasteSiteId'] == tankID) & (df['WastePhase'] == WastePhase),['Analyte']].values[:,0]
    elements_dict = dict(zip(elements,np.zeros(len(elements))))
    for analyte in analytes:
        if analyte in elements:
            elements_dict[analyte] += df.loc[(df['WasteSiteId'] == tankID) & (df['WastePhase'] == WastePhase)
                                            & (df['Analyte'] == analyte),['Mass (kg)']].values[0,0]
    return elements_dict


'''
Takes in the total excel waste data, tank ID, desired waste phase, and
outputs the mass of each indiviudally labled nuclide per tank per waste phase as a dictionary.
The nuclides are reformatted so that the element symbol is first followed by the mass number to make it easier to put into openmc.
'''
def nuclide_masses_per_tank_waste_phase(data,tankID,WastePhase):
    analytes = df.loc[(df['WasteSiteId'] == tankID) & (df['WastePhase'] == WastePhase),['Analyte']].values[:,0]
    nuclides = []
    nuclides_reformatted = []
    masses = []
    for i in range(len(analytes)):
        if analytes[i][0].isdigit():
           if ((analytes[i][1].isdigit()) or ((analytes[i][1].isalpha()))):
               nuclides.append(analytes[i])
               masses.append(df.loc[(df['WasteSiteId'] == tankID) & (df['WastePhase'] == WastePhase)& (df['Analyte'] == analytes[i]),['Mass (kg)']].values[0,0])
    nuclides_mass_dict = dict(zip(nuclides,masses))
    for analyte in analytes:
        if analyte[0].isdigit():
            digits = ''
            letters = ''
            new_key = ''
            for i in range(len(analyte)):
                if analyte[0].isdigit():
                    if ((analyte[1].isdigit()) or ((analyte[1].isalpha()))):
                        if analyte[i].isdigit():
                                digits += analyte[i]
                        if analyte[i].isalpha():
                            if analyte[i] == 'm':
                                if analyte[i-1].isdigit():
                                    digits += analyte[i]
                                else:
                                    letters += analyte[i]
                            else:
                                letters += analyte[i]
            new_key = letters+digits
            nuclides_reformatted.append(new_key)
    nuclides_reformatted_dict = dict(zip(nuclides_reformatted,nuclides_mass_dict.values()))
    return nuclides_reformatted_dict

def compounds_in_tank_list(data,tankID,WastePhase,compounds):
    analytes = df.loc[(df['WasteSiteId'] == tankID) & (df['WastePhase'] == WastePhase),['Analyte']].values[:,0]
    compounds_present = []
    for analyte in analytes:
        if analyte in compounds:
            compounds_present.append(analyte)
    return compounds_present

def elements_from_nuclides(data,tankID,WastePhase):
    nuclides = nuclide_masses_per_tank_waste_phase(data,tankID,WastePhase)
    elements_separated = []
    for compounds in nuclides.keys():
        letters = ''
        for i in range(len(compounds)):
            sep_element = ''
            if i < 2:
                if compounds[i].isalpha():
                    letters += compounds[i]
        sep_element = letters
        elements_separated.append(sep_element)
        elements_separated_dict = dict(zip(elements_separated,nuclides.keys()))
    return elements_separated_dict
'''
must get objects returned from:
    nuclide_masses_per_tank_waste_phase()
    element_masses_per_tank_per_waste_phase()
    total_molecular_mass_per_molecule_dict()
    elements_from_nuclides()
updates compound mass and element mass accordingly to avoid double counting
'''
def subtracting_nuclide_mass_from_comp_or_elements(nuclides_present,elements_present,compounds_present,elements_from_nuclides):
    double_elements = ['C','H','O','P','N','Cl','F','S']
    for substance in elements_present.keys():
        if substance in double_elements:
            if substance in elements_from_nuclides.keys():
                elements_present[substance] -= nuclides_present[elements_from_nuclides[substance]]
            elif compounds_present[substance] > 1e-8:
                compounds_present[substance] -= nuclides_present[elements_from_nuclides[substance]]
    return [elements_present,compounds_present]

In [53]:
elements = ['Ag','Al','As','B','Ba','Be','Bi','Br','Ca','Cd','Ce','Cl','Co','Cr','Cu','Eu','F','Fe','K','La','Li',
            'Mg','Mn','Mo','Na','Nb','Nd','Ni','Pb']

In [7]:
color = ['red','yellow']
test = dict(zip(color,np.zeros(2)))
test

{'red': 0.0, 'yellow': 0.0}

In [8]:
df.loc[(df['WasteSiteId'] == '241-A-104') & (df['WastePhase'] == 'Sludge (Liquid & Solid)')
                                            & (df['Analyte'] == 'Al'),['Mass (kg)']].values[0,0]

6500.0

In [54]:
elements_dict = element_masses_per_tank_per_waste_phase(df,'241-A-104','Sludge (Liquid & Solid)',elements)
elements_dict

{'Ag': 50.3,
 'Al': 6500.0,
 'As': 5.38,
 'B': 0.0,
 'Ba': 188.0,
 'Be': 2.81,
 'Bi': 0.0,
 'Br': 0.0,
 'Ca': 1060.0,
 'Cd': 0.936,
 'Ce': 206.0,
 'Cl': 42.1,
 'Co': 230.0,
 'Cr': 260.0,
 'Cu': 161.0,
 'Eu': 0.0,
 'F': 22.4,
 'Fe': 28200.0,
 'K': 18.8,
 'La': 115.0,
 'Li': 0.0,
 'Mg': 307.0,
 'Mn': 5190.0,
 'Mo': 10.5,
 'Na': 10700.0,
 'Nb': 0.0,
 'Nd': 918.0,
 'Ni': 1310.0,
 'Pb': 1180.0}

In [10]:
a = ['r','-']
a[1].isalpha()

False

In [75]:
nuclides = nuclide_masses_per_tank_waste_phase(df,'241-A-101','Sludge (Liquid & Solid)')
nuclides

{'Ru106': 1.12e-15,
 'Cd113m': 6.283702482e-07,
 'Sb125': 6.03664416e-08,
 'Sn126': 0.001532011991,
 'I129': 0.0004435506713,
 'Cs134': 2.113003e-10,
 'Cs137': 0.01008238751,
 'Ba137m': 1.5357727e-09,
 'C14': 5.102497538e-06,
 'Sm151': 0.2292165583,
 'Eu152': 1.842150711e-05,
 'Eu154': 0.000618129326,
 'Eu155': 7.393068226e-05,
 'Ra226': 1.85e-10,
 'Ac227': 0.0,
 'Ra228': 5.77e-18,
 'Th229': 5.19e-12,
 'Pa231': 2.75505446e-08,
 'Th232': 1.43e-08,
 'U232': 0.0,
 'U233': 2.261809655e-07,
 'U234': 0.002479551749,
 'U235': 0.2984729292,
 'U236': 0.006137530147,
 'Np237': 0.004979359067,
 'Pu238': 0.0001173448538,
 'U238': 44.93245254,
 'Pu239': 0.7523763493,
 'Pu240': 0.05064294522,
 'Am241': 0.07467040019,
 'Pu241': 0.0004940788196,
 'Cm242': 6.57715406e-08,
 'Pu242': 0.0002109351203,
 'Am243': 0.0006912788659,
 'Cm243': 1.960086236e-07,
 'Cm244': 2.32236387e-06,
 'H3': 4.1677059e-09,
 'Ni59': 0.007462499373,
 'Co60': 2.750995135e-06,
 'Ni63': 0.0009110292692,
 'Se79': 0.0006004359118,
 '

In [76]:
nuclides = nuclide_masses_per_tank_waste_phase(df,'241-A-101','Sludge (Liquid & Solid)')
nuclides

{'Ru106': 1.12e-15,
 'Cd113m': 6.283702482e-07,
 'Sb125': 6.03664416e-08,
 'Sn126': 0.001532011991,
 'I129': 0.0004435506713,
 'Cs134': 2.113003e-10,
 'Cs137': 0.01008238751,
 'Ba137m': 1.5357727e-09,
 'C14': 5.102497538e-06,
 'Sm151': 0.2292165583,
 'Eu152': 1.842150711e-05,
 'Eu154': 0.000618129326,
 'Eu155': 7.393068226e-05,
 'Ra226': 1.85e-10,
 'Ac227': 0.0,
 'Ra228': 5.77e-18,
 'Th229': 5.19e-12,
 'Pa231': 2.75505446e-08,
 'Th232': 1.43e-08,
 'U232': 0.0,
 'U233': 2.261809655e-07,
 'U234': 0.002479551749,
 'U235': 0.2984729292,
 'U236': 0.006137530147,
 'Np237': 0.004979359067,
 'Pu238': 0.0001173448538,
 'U238': 44.93245254,
 'Pu239': 0.7523763493,
 'Pu240': 0.05064294522,
 'Am241': 0.07467040019,
 'Pu241': 0.0004940788196,
 'Cm242': 6.57715406e-08,
 'Pu242': 0.0002109351203,
 'Am243': 0.0006912788659,
 'Cm243': 1.960086236e-07,
 'Cm244': 2.32236387e-06,
 'H3': 4.1677059e-09,
 'Ni59': 0.007462499373,
 'Co60': 2.750995135e-06,
 'Ni63': 0.0009110292692,
 'Se79': 0.0006004359118,
 '

In [13]:
if ((not analytes[0][3].isdigit()) and (not analytes[0][3].isdigit())):
    print('False')

NameError: name 'analytes' is not defined

In [100]:
df.loc[(df['WasteSiteId'] == '241-A-104') & (df['WastePhase'] == 'Sludge (Liquid & Solid)') & (df['Analyte'] == 'Al'),['Mass (kg)']].values[0,0]

6500.0

In [89]:
compounds = ["1-Butanol","1,1-Dichloroethene","1,1,1-Trichloroethane","1,1,2-Trichloro-1,2,2-trifluoroethane","1,1,2-Trichloroethane",
             "1,1,2,2-Tetrachloroethane","1,2-Dichlorobenzene","1,2-Dichloroethane","1,2,4-Trichlorobenzene","1,4-Dichlorobenzene",
             "2-Butanone","2-Chlorophenol","2-Ethoxyethanol","2-Methylphenol","2-Nitrophenol","2-Nitropropane","2,4-Dinitrotoluene",
             "2,4,5-Trichlorophenol","2,4,6-Trichlorophenol","2,6-Bis(1,1-dimethylethyl)-4-methylphenol","4-Chloro-3-methylphenol",
             "4-Methyl-2-Pentanone","4-Nitrophenol","Acenaphthene","Acetate","Acetone","Aroclors(TotalPCB)","Benzene","Benzo(a)pyrene",
             "Bis(2-ethylhexyl)phthalate","Butylbenzylphthalate","Carbondisulfide","Carbontetrachloride","Chlorobenzene","Chloroform",
             "CN","Cresol","Cresol(m&p)","Cyclohexanone","Di-n-butylphthalate","Di-n-octylphthalate","Dibenz[a,h]anthracene",
             "Diethylphthalate","Diphenylamine","Ethylacetate","Ethylether","Ethylbenzene","Fluoranthene","Formate","FreeOH","Glycolate",
             "Hexachlorobenzene","Hexachlorobutadiene","Hexachloroethane","Hexone","Isobutanol","m-Cresol","Methylenechloride",
             "Morpholine,4-nitroso-","N-Nitroso-di-n-propylamine","N-Nitrosodimethylamine","Naphthalene","NH3","Nitrobenzene","NO2",
             "NO3","Oxalate","Pentachlorophenol","Phenol","PO4","Pyrene","Pyridine","SO4","Sulfide","Tetrachloroethene","Thiosulfate",
             "TlCasCO3","Toluene","Trans-1,3-Dichloropropene","Tributylphosphate","Trichloroethene","Trichlorofluoromethane",
             "Vinylchloride","Xylene(m&p)","Xylene(o)","Xylenes(total)"]

formulae = ["C4H10O","C2H2Cl2","C2H3Cl3","C2Cl3F3","C2H3Cl3","C2H2Cl4","C6H4Cl2","C2H4Cl2","C6H3Cl3","C6H4Cl2","C4H8O","C6H5ClO","C4H10O2",
            "C7H8O","C6H5NO3","C3H7NO2","C7H6N2O4","C6H3Cl3O","C6H3Cl3O","C15H24O","C7H7ClO","C6H12O","C6H5NO3","C12H10","C2H3O2","C3H6O",
            "C12H3.5Cl6.5","C6H6","C20H12","C24H38O4","C19H20O4","CS2","CCl4","C6H5Cl","CHCl3","CN","C7H8O","C7H8O","C6H10O","C16H22O4",
            "C24H38O4","C22H14","C12H14O4","C12H11N","C4H8O2","C4H10O","C8H10","C16H10","CHO2","OH","C2H3O3","C6Cl6","C4Cl6","C2Cl6",
            "C6H12O","C4H10O","C7H8O","CH2Cl2","C4H8N2O2","C6H14N2O","C2H6N2O","C10H8","NH3","C6H5NO2","NO2","NO3","C2O4","C6HCl5O","C6H6O",
            "PO4","C16H10","C5H5N","SO4","S2","C2Cl4","O3S2","Tl2CO3","C7H8","C3H4Cl2","C12H27O4P","C2HCl3","CCl3F","C2H3Cl","C8H10","C8H10","C8H10"]


###################################[c,h,o,p,n,cl,f,s,tl]
vec_Butanol =                      [4,10,1,0,0,0,0,0,0]
vec_Dichloroethene =               [2,2,0,0,0,2,0,0,0]
vec_Trichloroethane1 =             [2,3,0,0,0,3,0,0,0]
vec_Trichlorotrifluoroethane =     [2,0,0,0,0,3,3,0,0]
vec_Trichloroethane2 =             [2,3,0,0,0,3,0,0,0]
vec_Tetrachloroethane =            [2,2,0,0,0,4,0,0,0]
vec_Dichlorobenzene2 =             [6,4,0,0,0,2,0,0,0]
vec_Dichloroethane =               [2,4,0,0,0,2,0,0,0]
vec_Trichlorobenzene =             [6,3,0,0,0,3,0,0,0]
vec_Dichlorobenzene4 =             [6,4,0,0,0,2,0,0,0]
vec_Butanone =                     [4,8,1,0,0,0,0,0,0]
vec_Chlorophenol =                 [6,5,1,0,0,1,0,0,0]
vec_Ethoxyethanol =                [4,10,2,0,0,0,0,0,0]
vec_Methylphenol =                 [7,8,1,0,0,0,0,0,0]
vec_Nitrophenol2 =                 [6,5,3,0,1,0,0,0,0]
vec_Nitropropane =                 [3,7,2,0,1,0,0,0,0]
vec_Dinitrotoluene =               [7,6,4,0,2,0,0,0,0]
vec_Trichlorophenol5 =             [6,3,1,0,0,3,0,0,0]
vec_Trichlorophenol6 =             [6,3,1,0,0,3,0,0,0]
vec_Bisdimethylethylmethylphenol = [15,24,1,0,0,0,0,0,0]
vec_Chloromethylphenol =           [7,7,1,0,0,1,0,0,0]
vec_Methylpentanone =              [6,12,1,0,0,0,0,0,0]
vec_Nitrophenol4 =                 [6,5,3,0,1,0,0,0,0]
vec_Acenaphthene =                 [12,10,0,0,0,0,0,0,0]
vec_Acetate =                      [2,3,2,0,0,0,0,0,0]
vec_Acetone =                      [3,6,1,0,0,0,0,0,0]
vec_Aroclors =                     [12,3.5,0,0,0,6.5,0,0,0]
vec_Benzene =                      [6,6,0,0,0,0,0,0,0]
vec_Benzopyrene =                  [20,12,0,0,0,0,0,0,0]
vec_Bisethylhexylphthalate =       [24,38,4,0,0,0,0,0,0]
vec_Butylbenzylphthalate =         [19,20,4,0,0,0,0,0,0]
vec_Carbondisulfide =              [1,0,0,0,0,0,0,2,0]
vec_Carbontetrachloride =          [1,0,0,0,0,4,0,0,0]
vec_Chlorobenzene =                [6,5,0,0,0,1,0,0,0]
vec_Chloroform =                   [1,1,0,0,0,3,0,0,0]
vec_CN =                           [1,0,0,0,1,0,0,0,0]
vec_Cresol =                       [7,8,1,0,0,0,0,0,0]
vec_Cresolmp =                     [7,8,1,0,0,0,0,0,0]
vec_Cyclohexanone =                [6,10,1,0,0,0,0,0,0]
vec_Dibutylphthalate =             [16,22,4,0,0,0,0,0,0]
vec_Dioctylphthalate =             [24,38,4,0,0,0,0,0,0]
vec_Dibenzanthracene =             [22,14,0,0,0,0,0,0,0]
vec_Diethylphthalate =             [12,14,4,0,0,0,0,0,0]
vec_Diphenylamine =                [12,11,0,0,1,0,0,0,0]
vec_Ethylacetate =                 [4,8,2,0,0,0,0,0,0]
vec_Ethylether =                   [4,10,1,0,0,0,0,0,0]
vec_Ethylbenzene =                 [8,10,0,0,0,0,0,0,0]
vec_Fluoranthene =                 [16,10,0,0,0,0,0,0,0]
vec_Formate =                      [1,1,2,0,0,0,0,0,0]
vec_FreeOH =                       [0,1,1,0,0,0,0,0,0]
vec_Glycolate =                    [2,3,3,0,0,0,0,0,0]
vec_Hexachlorobenzene =            [6,0,0,0,0,6,0,0,0]
vec_Hexachlorobutadiene =          [4,0,0,0,0,6,0,0,0]
vec_Hexachloroethane =             [2,0,0,0,0,6,0,0,0]
vec_Hexone =                       [6,12,1,0,0,0,0,0,0]
vec_Isobutanol =                   [4,10,1,0,0,0,0,0,0]
vec_mCresol =                      [7,8,1,0,0,0,0,0,0]
vec_Methylenechloride =            [1,2,0,0,0,2,0,0,0]
vec_Morpholinenitroso =            [4,8,2,0,2,0,0,0,0]
vec_Nitrosodipropylamine =         [6,14,1,0,2,0,0,0,0]
vec_Nitrosodimethylamine =         [0,0,0,0,2,0,0,0,0]
vec_Naphthalene =                  [10,8,0,0,0,0,0,0,0]
vec_NH3 =                          [0,3,0,0,1,0,0,0,0]
vec_Nitrobenzene =                 [6,5,2,0,1,0,0,0,0]
vec_NO2 =                          [0,0,2,0,1,0,0,0,0]
vec_NO3 =                          [0,0,3,0,1,0,0,0,0]
vec_Oxalate =                      [2,0,4,0,0,0,0,0,0]
vec_Pentachlorophenol =            [6,1,1,0,0,5,0,0,0]
vec_Phenol =                       [6,6,1,0,0,0,0,0,0]
vec_PO4 =                          [0,0,4,1,0,0,0,0,0]
vec_Pyrene =                       [16,10,0,0,0,0,0,0,0]
vec_Pyridine =                     [5,5,0,0,1,0,0,0,0]
vec_SO4 =                          [0,0,4,0,0,0,0,1,0]
vec_Sulfide =                      [0,0,0,0,0,0,0,2,0]
vec_Tetrachloroethene =            [2,0,0,0,0,4,0,0,0]
vec_Thiosulfate =                  [0,0,3,0,0,0,0,2,0]
vec_TlCasCO3 =                     [1,0,3,0,0,0,0,0,2]
vec_Toluene =                      [7,8,0,0,0,0,0,0,0]
vec_Transdichloropropene =         [3,4,0,0,0,2,0,0,0]
vec_Tributylphosphate =            [12,27,4,1,0,0,0,0,0]
vec_Trichloroethene =              [2,1,0,0,0,3,0,0,0]
vec_Trichlorofluoromethane =       [1,0,0,0,0,3,1,0,0]
vec_Vinylchloride =                [2,3,0,0,0,1,0,0,0]
vec_Xylenemp =                     [8,10,0,0,0,0,0,0,0]
vec_Xyleneo =                      [8,10,0,0,0,0,0,0,0]
vec_Xylenet =                      [8,10,0,0,0,0,0,0,0]

mat_materials = np.stack((vec_Butanol,vec_Dichloroethene,vec_Trichloroethane1,vec_Trichlorotrifluoroethane,vec_Trichloroethane2,vec_Tetrachloroethane,
                          vec_Dichlorobenzene2,vec_Dichloroethane,vec_Trichlorobenzene,vec_Dichlorobenzene4,vec_Butanone,vec_Chlorophenol,
                          vec_Ethoxyethanol,vec_Methylphenol,vec_Nitrophenol2,vec_Nitropropane,vec_Dinitrotoluene,vec_Trichlorophenol5,
                          vec_Trichlorophenol6,vec_Bisdimethylethylmethylphenol,vec_Chloromethylphenol,vec_Methylpentanone,vec_Nitrophenol4,
                          vec_Acenaphthene,vec_Acetate,vec_Acetone,vec_Aroclors,vec_Benzene,vec_Benzopyrene,vec_Bisethylhexylphthalate,
                          vec_Butylbenzylphthalate,vec_Carbondisulfide,vec_Carbontetrachloride,vec_Chlorobenzene,vec_Chloroform,vec_CN,vec_Cresol,
                          vec_Cresolmp,vec_Cyclohexanone,vec_Dibutylphthalate,vec_Dioctylphthalate,vec_Dibenzanthracene,vec_Diethylphthalate,
                          vec_Diphenylamine,vec_Ethylacetate,vec_Ethylether,vec_Ethylbenzene,vec_Fluoranthene,vec_Formate,vec_FreeOH,vec_Glycolate,
                          vec_Hexachlorobenzene,vec_Hexachlorobutadiene,vec_Hexachloroethane,vec_Hexone,vec_Isobutanol,vec_mCresol,vec_Methylenechloride,
                          vec_Morpholinenitroso,vec_Nitrosodipropylamine,vec_Nitrosodimethylamine,vec_Naphthalene,vec_NH3,vec_Nitrobenzene,vec_NO2,
                          vec_NO3,vec_Oxalate,vec_Pentachlorophenol,vec_Phenol,vec_PO4,vec_Pyrene,vec_Pyridine,vec_SO4,vec_Sulfide,vec_Tetrachloroethene,
                          vec_Thiosulfate,vec_TlCasCO3,vec_Toluene,vec_Transdichloropropene,vec_Tributylphosphate,vec_Trichloroethene,
                          vec_Trichlorofluoromethane,vec_Vinylchloride,vec_Xylenemp,vec_Xyleneo,vec_Xylenet))


In [17]:
compounds_formula_dict = dict(zip(compounds,formulae))
compounds_formula_dict

{'1-Butanol': 'C4H10O',
 '1,1-Dichloroethene': 'C2H2Cl2',
 '1,1,1-Trichloroethane': 'C2H3Cl3',
 '1,1,2-Trichloro-1,2,2-trifluoroethane': 'C2Cl3F3',
 '1,1,2-Trichloroethane': 'C2H3Cl3',
 '1,1,2,2-Tetrachloroethane': 'C2H2Cl4',
 '1,2-Dichlorobenzene': 'C6H4Cl2',
 '1,2-Dichloroethane': 'C2H4Cl2',
 '1,2,4-Trichlorobenzene': 'C6H3Cl3',
 '1,4-Dichlorobenzene': 'C6H4Cl2',
 '2-Butanone': 'C4H8O',
 '2-Chlorophenol': 'C6H5ClO',
 '2-Ethoxyethanol': 'C4H10O2',
 '2-Methylphenol': 'C7H8O',
 '2-Nitrophenol': 'C6H5NO3',
 '2-Nitropropane': 'C3H7NO2',
 '2,4-Dinitrotoluene': 'C7H6N2O4',
 '2,4,5-Trichlorophenol': 'C6H3Cl3O',
 '2,4,6-Trichlorophenol': 'C6H3Cl3O',
 '2,6-Bis(1,1-dimethylethyl)-4-methylphenol': 'C15H24O',
 '4-Chloro-3-methylphenol': 'C7H7ClO',
 '4-Methyl-2-Pentanone': 'C6H12O',
 '4-Nitrophenol': 'C6H5NO3',
 'Acenaphthene': 'C12H10',
 'Acetate': 'C2H3O2',
 'Acetone': 'C3H6O',
 'Aroclors(TotalPCB)': 'C12H3.5Cl6.5',
 'Benzene': 'C6H6',
 'Benzo(a)pyrene': 'C20H12',
 'Bis(2-ethylhexyl)phthalate': 

In [19]:
compounds_in_tank = compounds_in_tank_list(df,'241-A-101','Sludge (Liquid & Solid)',compounds)
compounds_in_tank

['NO2', 'NO3', 'Oxalate', 'PO4', 'SO4']

In [21]:
compounds_in_tank_dict = dict(zip(compounds_in_tank,np.zeros(len(compounds_in_tank))))
compounds_in_tank_dict

{'NO2': 0.0, 'NO3': 0.0, 'Oxalate': 0.0, 'PO4': 0.0, 'SO4': 0.0}

In [24]:
for compound in compounds_in_tank_dict:
    if compound in compounds_formula_dict:
        compounds_in_tank_dict[compound] = compounds_formula_dict[compound]
compounds_in_tank_dict

{'NO2': 'NO2', 'NO3': 'NO3', 'Oxalate': 'C2O4', 'PO4': 'PO4', 'SO4': 'SO4'}

In [102]:
elements_in_tank = element_masses_per_tank_per_waste_phase(df,'241-A-104','Sludge (Liquid & Solid)',elements)
elements_in_tank

{'Ag': 50.3,
 'Al': 6500.0,
 'As': 5.38,
 'B': 0.0,
 'Ba': 188.0,
 'Be': 2.81,
 'Bi': 0.0,
 'Br': 0.0,
 'Ca': 1060.0,
 'Cd': 0.936,
 'Ce': 206.0,
 'Cl': 42.1,
 'Co': 230.0,
 'Cr': 260.0,
 'Cu': 161.0,
 'Eu': 0.0,
 'F': 22.4,
 'Fe': 28200.0,
 'K': 18.8,
 'La': 115.0,
 'Li': 0.0,
 'Mg': 307.0,
 'Mn': 5190.0,
 'Mo': 10.5,
 'Na': 10700.0,
 'Nb': 0.0,
 'Nd': 918.0,
 'Ni': 1310.0,
 'Pb': 1180.0}

In [30]:
test = ['C','H','O','P','N','Cl','F','S']
test_string = 'C2H3Cl'
molecules_in_compounds = []
for compound in test:
    if compound in test_string:
        if not compound in molecules_in_compounds:
            molecules_in_compounds.append(compound)
molecules_in_compounds

['C', 'H', 'Cl']

In [32]:
molecule_mass_from_compounds = total_molecular_mass_per_molecule_dict(df,'241-A-101','Sludge (Liquid & Solid)',compounds,mat_materials)
molecule_mass_from_compounds

{'C': 24.022,
 'H': 0.0,
 'O': 271.983,
 'P': 30.973761998,
 'N': 28.014,
 'Cl': 0.0,
 'F': 0.0,
 'S': 32.06,
 'Tl': 0.0}

In [77]:
nuclides

{'Ru106': 1.12e-15,
 'Cd113m': 6.283702482e-07,
 'Sb125': 6.03664416e-08,
 'Sn126': 0.001532011991,
 'I129': 0.0004435506713,
 'Cs134': 2.113003e-10,
 'Cs137': 0.01008238751,
 'Ba137m': 1.5357727e-09,
 'C14': 5.102497538e-06,
 'Sm151': 0.2292165583,
 'Eu152': 1.842150711e-05,
 'Eu154': 0.000618129326,
 'Eu155': 7.393068226e-05,
 'Ra226': 1.85e-10,
 'Ac227': 0.0,
 'Ra228': 5.77e-18,
 'Th229': 5.19e-12,
 'Pa231': 2.75505446e-08,
 'Th232': 1.43e-08,
 'U232': 0.0,
 'U233': 2.261809655e-07,
 'U234': 0.002479551749,
 'U235': 0.2984729292,
 'U236': 0.006137530147,
 'Np237': 0.004979359067,
 'Pu238': 0.0001173448538,
 'U238': 44.93245254,
 'Pu239': 0.7523763493,
 'Pu240': 0.05064294522,
 'Am241': 0.07467040019,
 'Pu241': 0.0004940788196,
 'Cm242': 6.57715406e-08,
 'Pu242': 0.0002109351203,
 'Am243': 0.0006912788659,
 'Cm243': 1.960086236e-07,
 'Cm244': 2.32236387e-06,
 'H3': 4.1677059e-09,
 'Ni59': 0.007462499373,
 'Co60': 2.750995135e-06,
 'Ni63': 0.0009110292692,
 'Se79': 0.0006004359118,
 '

In [34]:
nuclides.items()

TypeError: dict.items() takes no arguments (1 given)

In [78]:
double_elements = ['C','H','O','P','N','Cl','F','S']
for compounds in nuclides.keys():
    for element in double_elements:
        if element in compounds:
            print(compounds,nuclides[compounds])

Cd113m 6.283702482e-07
Sb125 6.03664416e-08
Sn126 0.001532011991
Cs134 2.113003e-10
Cs137 0.01008238751
C14 5.102497538e-06
Sm151 0.2292165583
Pa231 2.75505446e-08
Np237 0.004979359067
Pu238 0.0001173448538
Pu239 0.7523763493
Pu240 0.05064294522
Pu241 0.0004940788196
Cm242 6.57715406e-08
Pu242 0.0002109351203
Cm243 1.960086236e-07
Cm244 2.32236387e-06
H3 4.1677059e-09
Ni59 0.007462499373
Co60 2.750995135e-06
Ni63 0.0009110292692
Se79 0.0006004359118
Sr90 0.3245671231
Nb93m 9.213501968e-07


In [82]:
elements_separated = []
for compounds in nuclides.keys():
    letters = ''
    for i in range(len(compounds)):
        sep_element = ''
        if i < 2:
            if compounds[i].isalpha():
                letters += compounds[i]
    sep_element = letters
    elements_separated.append(sep_element)
    elements_separated_dict = dict(zip(elements_separated,nuclides.keys()))
elements_separated_dict

{'Ru': 'Ru106',
 'Cd': 'Cd113m',
 'Sb': 'Sb125',
 'Sn': 'Sn126',
 'I': 'I129',
 'Cs': 'Cs137',
 'Ba': 'Ba137m',
 'C': 'C14',
 'Sm': 'Sm151',
 'Eu': 'Eu155',
 'Ra': 'Ra228',
 'Ac': 'Ac227',
 'Th': 'Th232',
 'Pa': 'Pa231',
 'U': 'U238',
 'Np': 'Np237',
 'Pu': 'Pu242',
 'Am': 'Am243',
 'Cm': 'Cm244',
 'H': 'H3',
 'Ni': 'Ni63',
 'Co': 'Co60',
 'Se': 'Se79',
 'Sr': 'Sr90',
 'Y': 'Y90',
 'Nb': 'Nb93m',
 'Zr': 'Zr93',
 'Tc': 'Tc99'}

In [50]:
for elements in elements_separated:
    if elements in double_elements:
        print(elements)

C
H


In [101]:
compounds_241_A_104 = total_molecular_mass_per_molecule_dict(df,'241-A-103','Sludge (Liquid & Solid)',compounds,mat_materials)
compounds_241_A_104

{'C': 24.022,
 'H': 0.0,
 'O': 271.983,
 'P': 30.973761998,
 'N': 28.014,
 'Cl': 0.0,
 'F': 0.0,
 'S': 32.06,
 'Tl': 0.0}

In [66]:
if compounds_241_A_104['F'] < 1e-8:
    print('is zero')
else:
    print('not zero')

is zero


In [81]:
for substance in elements_in_tank.keys():
    if substance in double_elements:
        print(substance)
        

Cl
F


In [84]:
nuclides_241A104 = elements_from_nuclides(df,'241-A-104','Sludge (Liquid & Solid)')
nuclides_241A104

{'Ru': 'Ru106',
 'Cd': 'Cd113m',
 'Sb': 'Sb125',
 'Sn': 'Sn126',
 'I': 'I129',
 'Cs': 'Cs137',
 'Ba': 'Ba137m',
 'C': 'C14',
 'Sm': 'Sm151',
 'Eu': 'Eu155',
 'Ra': 'Ra228',
 'Ac': 'Ac227',
 'Th': 'Th232',
 'Pa': 'Pa231',
 'U': 'U238',
 'Np': 'Np237',
 'Pu': 'Pu242',
 'Am': 'Am243',
 'Cm': 'Cm244',
 'H': 'H3',
 'Ni': 'Ni63',
 'Co': 'Co60',
 'Se': 'Se79',
 'Sr': 'Sr90',
 'Y': 'Y90',
 'Nb': 'Nb93m',
 'Zr': 'Zr93',
 'Tc': 'Tc99'}

In [103]:
for substance in elements_in_tank.keys():
    if substance in double_elements:
        if substance in nuclides_241A104.keys():
            elements_in_tank[substance] -= nuclides[nuclides_241A104[substance]]
        elif compounds_241_A_104[substance] > 1e-8:
            compounds_241_A_104[substance] -= nuclides[nuclides_241A104[substance]]

In [104]:
elements_in_tank

{'Ag': 50.3,
 'Al': 6500.0,
 'As': 5.38,
 'B': 0.0,
 'Ba': 188.0,
 'Be': 2.81,
 'Bi': 0.0,
 'Br': 0.0,
 'Ca': 1060.0,
 'Cd': 0.936,
 'Ce': 206.0,
 'Cl': 42.1,
 'Co': 230.0,
 'Cr': 260.0,
 'Cu': 161.0,
 'Eu': 0.0,
 'F': 22.4,
 'Fe': 28200.0,
 'K': 18.8,
 'La': 115.0,
 'Li': 0.0,
 'Mg': 307.0,
 'Mn': 5190.0,
 'Mo': 10.5,
 'Na': 10700.0,
 'Nb': 0.0,
 'Nd': 918.0,
 'Ni': 1310.0,
 'Pb': 1180.0}

In [105]:
compounds_241_A_104

{'C': 24.022,
 'H': 0.0,
 'O': 271.983,
 'P': 30.973761998,
 'N': 28.014,
 'Cl': 0.0,
 'F': 0.0,
 'S': 32.06,
 'Tl': 0.0}

In [127]:
tank = '241-TX-101'
WastePhase = 'Saltcake Interstitial Liquid'
nuclides_tank = nuclide_masses_per_tank_waste_phase(df,tank,WastePhase)
elements_tank = element_masses_per_tank_per_waste_phase(df,tank,WastePhase,elements)
compounds_tank = total_molecular_mass_per_molecule_dict(df,tank,WastePhase,compounds,mat_materials)
elements_from_nuclides = elements_from_nuclides(df,tank,WastePhase)

In [128]:
elements_tank

{'Ag': 0.0,
 'Al': 302.0,
 'As': 0.0,
 'B': 0.0,
 'Ba': 0.0,
 'Be': 0.0,
 'Bi': 0.261,
 'Br': 0.0,
 'Ca': 0.505,
 'Cd': 0.0,
 'Ce': 0.0,
 'Cl': 75.6,
 'Co': 0.0,
 'Cr': 13.8,
 'Cu': 0.0,
 'Eu': 0.0,
 'F': 2.74,
 'Fe': 0.206,
 'K': 55.4,
 'La': 1.2e-10,
 'Li': 0.0,
 'Mg': 0.0,
 'Mn': 0.0495,
 'Mo': 0.0,
 'Na': 1890.0,
 'Nb': 0.0,
 'Nd': 0.0,
 'Ni': 0.233,
 'Pb': 0.514}

In [129]:
compounds_tank

{'C': 24.022,
 'H': 0.0,
 'O': 271.983,
 'P': 30.973761998,
 'N': 28.014,
 'Cl': 0.0,
 'F': 0.0,
 'S': 32.06,
 'Tl': 0.0}

In [130]:
elements_components = subtracting_nuclide_mass_from_comp_or_elements(nuclides_tank,elements_tank,compounds_tank,elements_from_nuclides)

In [140]:
len(elements_components)

2

In [133]:
elements_components[1]

{'C': 24.022,
 'H': 0.0,
 'O': 271.983,
 'P': 30.973761998,
 'N': 28.014,
 'Cl': 0.0,
 'F': 0.0,
 'S': 32.06,
 'Tl': 0.0}